In [1]:
import bql
import bqplot as bqp
import ipydatagrid as ipd
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import time

from datetime import datetime, timedelta
from IPython.display import display, clear_output
from plotly.subplots import make_subplots

In [2]:
# Connect to BQL
bq = bql.Service()

### Import Data from Coverage List

In [3]:
# Define the file path
file_path = 'altd_coverage_list_20250319.xlsx'

muni_pattern = r'^\d.*US Equity$'

# Read the 'coverage_list' sheet, skipping the first 5 rows
coverage_list = pd.read_excel(file_path, sheet_name='ALTD Coverage', skiprows=5).dropna()
muni_removal = ~coverage_list['Ticker'].str.match(muni_pattern, na=False)
coverage_list = coverage_list[muni_removal]
public_coverage = coverage_list[(coverage_list['Ownership'] == 'Public')&(coverage_list['Bloomberg Second Measure'] == 'Y')]
possible_tickers = public_coverage.Ticker.to_list()

# Read the 'brand_mapping' sheet, skipping the first 5 rows
brand_mapping = pd.read_excel(file_path, sheet_name='ALTD Coverage Brands', skiprows=5).dropna()
public_brands = brand_mapping[brand_mapping['Ticker'].isin(possible_tickers)]

In [4]:
# remove_list = []
# for ticker in possible_tickers:
#     brand_list = public_brands[public_brands['Ticker'] == i]['Brand'].to_list()
 
#     # Define the date
#     ref_date = '-10d'
    
#     # Define the target data
#     for brand in brand_list
#         data_items[brand] = {
#             brand: bq.data.average_transaction_value(
#                 dates=ref_date,
#                 brand='Shoes.com'
#             )
#         }
    
#     # Build and execute the BQL query
#     request = bql.Request(ticker, data_items)
#     response = bq.execute(request)
    
#     # Present the data as a Pandas DataFrame
#     df = response[0].df()
    

In [5]:
# public_brands.head()

### Data Retreival Functions

In [6]:
def get_data(universe, data_items, with_params=None, preferences=None):
    """Requests data from BQL and returns a DataFrame."""
    request = bql.Request(universe, data_items, with_params=with_params, preferences=preferences)
    response = bq.execute(request)
    df = pd.concat([data_item.df() for data_item in response], axis=1)
    return df

In [7]:
def get_transaction_data(universe, brands=[], channel_breakout=True):
    """Returns quarterly history of observed sales data."""

    date_range = bq.func.range('-30', '0')
    
    # Create a dictionary to hold the data items for each brand
    data_items = {}
    if channel_breakout:
        brands = ['Online','In_Store']
        safe_brands = brands
        for brand in brands:
            # Create the observed_sales data item for this brand
            data_items[brand] = bq.data.observed_sales(
                channel=brand,
                fa_period_offset=date_range,
                fa_period_type='Q'
            )
    else:
        safe_brands = []
        for brand in brands:
            # Escape any single quotes in the brand name
            safe_brand = brand.replace("'", "\\'")
            safe_brands.append(safe_brand)
            # Create the observed_sales data item for this brand
            data_items[safe_brand] = bq.data.observed_sales(
                brand=safe_brand,
                fa_period_offset=date_range,
                fa_period_type='Q'
            )
    
    request = bql.Request(universe, data_items)
    response = bq.execute(request)

    

    transaction_data = get_data(universe, data_items)

    if channel_breakout:
        transaction_data = transaction_data.loc[:, ~transaction_data.columns.duplicated()].drop(columns=['DATE','SOURCE','CURRENCY','CHANNEL'])
    else:
        transaction_data = transaction_data.loc[:, ~transaction_data.columns.duplicated()].drop(columns=['DATE','SOURCE','CURRENCY','BRAND'])
    transaction_data['All'] = transaction_data[[brand for brand in safe_brands]].sum(axis=1)

    share = pd.DataFrame()
    share['PERIOD_END_DATE'] = transaction_data['PERIOD_END_DATE']
    for brand in safe_brands:
        share[brand] = transaction_data[brand] / transaction_data['All']

    yoy = transaction_data.set_index('PERIOD_END_DATE').copy().pct_change(periods=4).dropna(how='all').reset_index()

    return share, yoy

#together_total, together_pct = get_transaction_data('NFLX US Equity')

In [8]:
# Example usage:
# share, yoy = get_transaction_data(
#     universe='LYFT US Equity', 
#     brands=["Walmart", "Sam's Club", "Walmart Gas", "Shoes.com"],
#     channel_breakout=True
# )
# share

In [9]:
def get_brand_data(universe, brands=[], channel_breakout=True):
    # Define the date range for the last ~545 days
    date_range = bq.func.range('-545D', '0D')
    
    # Create a dictionary to hold the data items for each brand
    data_items = {}

    no_data_list = []
    if channel_breakout:
        brands = ['Online','In_Store']
        for brand in brands.copy():
            # Create the observed_sales data item for this brand
            data_item={}
            data_item['Test'] = bq.data.observed_sales(
                channel=brand,
                dates='-10d'
            )
            request = bql.Request(universe, data_item)
            response = bq.execute(request)
            df = response[0].df()

            if np.isnan(df.iloc[0]['Test']):
                no_data_list.append(brand)
                brands.remove(brand)
            else:
                data_items[brand] = bq.data.observed_sales(
                    channel=brand,
                    dates=date_range
                )
    else:
        for brand in brands.copy():
            # Escape any single quotes in the brand name
            data_item={}
            safe_brand = brand.replace("'", "\\'")
            data_item['Test'] = bq.data.observed_sales(
                brand=safe_brand,
                dates='-10d'
            )
            request = bql.Request(universe, data_item)
            response = bq.execute(request)
            df = response[0].df()

            if np.isnan(df.iloc[0]['Test']):
                no_data_list.append(brand)
                brands.remove(brand)
            else:
                data_items[brand] = bq.data.observed_sales(
                    brand=safe_brand,
                    dates=date_range
                )

    # Create and execute the request for the specified universe and data items
    request = bql.Request(universe, data_items)
    response = bq.execute(request)
    
    # Concatenate the resulting DataFrames side by side
    df = pd.concat([item.df() for item in response], axis=1)

    # Step 1: Identify positions of 'PERIOD_END_DATE' columns that are entirely NaT.
    cols_to_drop = [i for i, col in enumerate(df.columns)
                    if col == 'PERIOD_END_DATE' and df.iloc[:, i].isna().all()]
    
    # Instead of dropping by label (which would remove all columns with that name),
    # rebuild the dataframe selecting only the columns not in cols_to_drop.
    cols_to_keep = [i for i in range(df.shape[1]) if i not in cols_to_drop]
    df = df.iloc[:, cols_to_keep]
    
    # Remove any duplicated columns
    df = df.loc[:, ~df.columns.duplicated()]
    
    # Drop unnecessary columns (like SOURCE, BRAND, DATE, PER, and CURRENCY)
    columns_to_drop = [col for col in ['SOURCE', 'BRAND', 'CHANNEL'] if col in df.columns]
    daily_data = df.drop(columns=columns_to_drop).drop(['DATE', 'PER', 'CURRENCY'], axis=1)
    
    # Retrieve quarter metadata (sales_rev_turn returns a dict-like series for each)
    current_q_start = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='0'
    )['PERIOD_END_DATE']
    
    # Unreported quarter end date
    current_q_end = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='1'
    )['PERIOD_END_DATE']
    
    # Comparison quarter start date
    comparison_q_start = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='-4'
    )['PERIOD_END_DATE']
    
    # Comparison quarter end date
    comparison_q_end = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='-3'
    )['PERIOD_END_DATE']
    
    data_items_meta = {
        'Current Quarter Start': current_q_start,
        'Current Quarter End': current_q_end,
        'Comparison Quarter Start': comparison_q_start,
        'Comparison Quarter End': comparison_q_end,
        'Total Days in Current Quarter': current_q_end - current_q_start,
        'Days in Comp Quarter': comparison_q_end - comparison_q_start
    }
    
    with_params = {
        'act_est_mapping': 'precise',
        'filing_status': 'MRC'
    }
    
    request = bql.Request(universe, data_items_meta, with_params=with_params, preferences=None)
    company_data = bq.execute(request)
    
    quarter_meta = pd.concat([data_item.df() for data_item in company_data], axis=1)
    
    # Extract date boundaries from quarter_meta (assuming one row)
    curr_start = pd.to_datetime(quarter_meta['Current Quarter Start'].iloc[0])
    curr_end   = pd.to_datetime(quarter_meta['Current Quarter End'].iloc[0])
    comp_start = pd.to_datetime(quarter_meta['Comparison Quarter Start'].iloc[0])
    comp_end   = pd.to_datetime(quarter_meta['Comparison Quarter End'].iloc[0])
    
    # Filter the DataFrame into current and comparison quarter data
    curr_quarter = daily_data[(daily_data['PERIOD_END_DATE'] >= curr_start) & 
                              (daily_data['PERIOD_END_DATE'] <= curr_end)].copy()
    comp_quarter = daily_data[(daily_data['PERIOD_END_DATE'] >= comp_start) & 
                              (daily_data['PERIOD_END_DATE'] <= comp_end)].copy()
    
    quarter_meta['Days Elapsed in Current Quarter'] = len(curr_quarter)
    
    # Identify numeric columns (the sales/brand data)
    brand_cols = curr_quarter.select_dtypes(include=[np.number]).columns.tolist()
    
    # Compute cumulative sums for each brand (QTD values)
    for col in brand_cols:
        curr_quarter[col] = curr_quarter[col].cumsum()
        comp_quarter[col] = comp_quarter[col].cumsum()
    
    # Reset the index so we can merge on the row number
    curr_quarter = curr_quarter.reset_index(drop=True)
    comp_quarter = comp_quarter.reset_index(drop=True)
    
    # Merge the current and comparison quarters on the index (row number)
    merged = pd.merge(
        curr_quarter, comp_quarter,
        left_index=True, right_index=True,
        suffixes=('_curr', '_comp'),
        how='outer'
    )
    
    # Compute the QTD year-over-year growth for each brand.
    # For example: Walmart_YoY_QTD = (Walmart_curr / Walmart_comp) - 1
    for brand in brands:
        curr_col = brand + '_curr'
        comp_col = brand + '_comp'
        merged[brand + '_YoY_QTD'] = merged[curr_col] / merged[comp_col] - 1

    # Compute the "All" column: total QTD current sales divided by total QTD comp sales, minus one.
    merged['All_YoY_QTD'] = (merged[[brand + '_curr' for brand in brands]].sum(axis=1) / 
                             merged[[brand + '_comp' for brand in brands]].sum(axis=1)) - 1
    
    # Rename the current quarter date column for clarity
    merged.rename(columns={'PERIOD_END_DATE_curr': 'PERIOD_END_DATE'}, inplace=True)
    
    # Set the index to be PERIOD_END_DATE
    merged.set_index('PERIOD_END_DATE', inplace=True)
    
    # Drop all columns that do not end in 'YoY_QTD'
    merged_growth = merged[[col for col in merged.columns if col.endswith('YoY_QTD')]].copy()
    
    # Rename columns to drop the '_YoY_QTD' suffix so they are just the brand names.
    # This will also rename 'All_YoY_QTD' to 'All'.
    merged_growth.rename(columns=lambda x: x.replace('_YoY_QTD', ''), inplace=True)
    merged_growth.dropna(subset=brand_cols, how='all',inplace=True)
    merged_growth.reset_index(inplace=True)
    
    share = pd.DataFrame()
    # Compute share of sales: first calculate total current quarter sales per day
    share['Total'] = curr_quarter.set_index('PERIOD_END_DATE').sum(axis=1)
    for brand in brands:
        share[brand] = curr_quarter.set_index('PERIOD_END_DATE')[brand] / share['Total']
    share.drop(columns=['Total'], inplace=True)
    share.reset_index(inplace=True)
    
    return merged_growth, share, quarter_meta
    #return brands, no_data_list

In [10]:
# Example usage:
# curr_quarter, dl = get_brand_data(
#     universe='WMT US Equity', 
#     brands=["Bare Necessities", "Foo", "Walmart", "Sam's Club", "Walmart Gas", "Shoes.com"],
#     channel_breakout=False
# )
# curr_quarter

In [11]:
# Example usage:
# merged_growth, share, quarter_meta = get_brand_data(
#     universe='WMT US Equity', 
#     brands=["Bare Necessities", "Bonobos", "Walmart", "Sam's Club", "Walmart Gas", "Shoes.com"],
#     channel_breakout=False
# )
# merged_growth

## Working

In [12]:
# Assume these are already defined:
# possible_tickers, get_company_data(universe), get_transaction_data(universe), public_brands
# Also assume get_brand_data(universe, brands, channel_breakout) is defined appropriately.

# Global widgets for existing app.
status = widgets.HTML(value='')       # Spinner/status widget.
top_out = widgets.Output()            # For quarter progress HTML widget.
left_out = widgets.Output()           # For left column charts.
right_out = widgets.Output()          # For right column charts.

# New widgets for breakout selection.
breakout_dropdown = widgets.Dropdown(
    options=['Channel', 'Brands'],
    description='Breakout By:',
    value='Channel'
)
breakout_out = widgets.Output()       # For displaying brand checkboxes when "Brands" is selected.

# Global variables to store the brand checkboxes and selected brands.
brand_checkboxes = []
selected_brands = []

channel_breakout = True
brands = []

def update_breakout(ticker):
    """Update the breakout output widget based on breakout_dropdown.
       If 'Brands' is selected, filter public_brands for the given ticker and display
       checkboxes arranged in columns (max 5 rows per column). All checkboxes are initially selected.
       The global variable 'selected_brands' is updated with the list of selected brand names."""
    global brand_checkboxes, selected_brands
    with breakout_out:
        clear_output(wait=True)
        if breakout_dropdown.value == 'Brands':
            # Filter public_brands for the selected ticker and get unique brands.
            brands_list = public_brands[public_brands['Ticker'] == ticker]['Brand'].unique()
            # Create a list of checkboxes (one per brand), all selected by default.
            brand_checkboxes = [widgets.Checkbox(value=True, description=brand) for brand in brands_list]
            
            # Define a callback to update selected_brands when any checkbox changes.
            def update_selected_brands(change):
                global selected_brands
                selected_brands = [cb.description for cb in brand_checkboxes if cb.value]
            
            # Attach the observer to each checkbox.
            for cb in brand_checkboxes:
                cb.observe(update_selected_brands, names='value')
            
            # Helper function to chunk the list into groups of 5.
            def chunks(lst, n):
                return [lst[i:i+n] for i in range(0, len(lst), n)]

            # Create button to clear all checkbox selections.
            
            # Create a list of VBox containers—each contains up to 5 checkboxes.
            cols = [widgets.VBox(chunk) for chunk in chunks(brand_checkboxes, 5)]
            # Arrange the columns side by side in an HBox.
            #grid = widgets.HBox([unselect_button] + cols)
            grid.children = [unselect_button] + cols
            display(grid)
            # Initialize selected_brands based on default values.
            selected_brands = [cb.description for cb in brand_checkboxes if cb.value]
        else:
            clear_output(wait=True)
            #grid = widgets.HBox()
            grid.children = []

def update_plots(ticker):
    global channel_breakout, brands
    universe = ticker

    # Set channel_breakout and brands based on breakout_dropdown.
    if breakout_dropdown.value == 'Brands':
        channel_breakout = False
        brands = selected_brands
    else:
        channel_breakout = True
        brands = []
    
    # Get the data.
    final_df, share, quarter_metadata = get_brand_data(universe, brands=brands, channel_breakout=channel_breakout)
    # Drop duplicate "PERIOD_END_DATE" columns if they exist.
    final_df = final_df.loc[:, ~final_df.columns.duplicated()]
    together_total, together_pct = get_transaction_data(universe, brands=brands, channel_breakout=channel_breakout)
    
    # Create the left column figures.
    fig1 = px.line(
        final_df[7:],
        x='PERIOD_END_DATE',
        y=final_df.columns[1:],
        template='plotly_dark',
        title=f'{dropdown.value} - QTD Breakout',
        labels={'value': 'Y/Y Observed Sales', 'variable': ''}
    )
    fig1.update_layout(yaxis=dict(tickformat=".1%"))
    fig1.update_xaxes(title_text="Days into Quarter")
    
    fig2 = px.area(
        share[7:],
        x='PERIOD_END_DATE',
        y=share.columns[1:],
        template='plotly_dark',
        title=f'{dropdown.value} - QTD Share of Observed Sales',
        labels={'value': 'Share (%)', 'variable': ''}
    )
    fig2.update_layout(yaxis=dict(tickformat=".1%"))
    fig2.update_xaxes(title_text="Days into Quarter")
    
    # Create the right column figures.
    fig3 = px.line(
        together_pct,
        x='PERIOD_END_DATE',
        y=together_pct.columns[1:],
        template='plotly_dark',
        title=f'{dropdown.value} - Fiscal Quarter Y/Y Growth Breakout',
        labels={'value': 'Y/Y Observed Sales', 'variable': ''}
    )
    fig3.update_layout(yaxis=dict(tickformat=".1%"))
    fig3.update_xaxes(title_text="Fiscal Period End Date")
    
    fig4 = px.area(
        together_total,
        x='PERIOD_END_DATE',
        y=together_total.columns[1:],
        template='plotly_dark',
        title=f'{dropdown.value} - Fiscal Quarterly Share of Observed Sales',
        labels={'value': 'Share (%)', 'variable': ''}
    )
    fig4.update_layout(yaxis=dict(tickformat=".1%"))
    fig4.update_xaxes(title_text="Fiscal Period End Date")
    
    # Function to rename legends and update "All" traces.
    def rename_legends(fig):
        for trace in fig.data:
            name_lower = trace.name.lower()
            if name_lower in ["all", "qtd - all", "total"]:
                trace.name = "All"
                trace.legendgroup = "All"
                trace.update(line=dict(dash='dash', width=4, color='white'))
    rename_legends(fig1)
    # rename_legends(fig2)
    rename_legends(fig3)
    # rename_legends(fig4)
    
    # Build the quarter progress HTML widget using rounded values.
    row_val = quarter_metadata.iloc[0]
    progress_html = (
        f"Quarter Progress: {int(round(row_val['Days Elapsed in Current Quarter']))} / "
        f"{int(round(row_val['Total Days in Current Quarter']))}<br>"
        f"Comp Quarter Length: {int(round(row_val['Days in Comp Quarter']))}"
    )
    progress_widget = widgets.HTML(value=progress_html)
    
    # Update the top output with the progress HTML.
    with top_out:
        clear_output(wait=True)
        display(progress_widget)
    
    # Update left and right outputs with the corresponding figures.
    with left_out:
        clear_output(wait=True)
        fig1.show()
        fig2.show()
    with right_out:
        clear_output(wait=True)
        fig3.show()
        fig4.show()
    
    # Once finished, clear the status spinner.
    status.value = ''

grid=widgets.HBox()
# Create a new "Run" button that triggers update_plots.
run_button = widgets.Button(
    description="Run",
    button_style=''  # You can set 'primary' if you like a colored button.
)

def on_run_button_click(b):
    # When the button is clicked, set spinner, then update plots using the selected ticker.
    status.value = '<div style="text-align:center;"><i class="fa fa-spinner fa-spin" style="font-size:24px;"></i> Loading...</div>'
    time.sleep(0.1)
    update_plots(dropdown.value)

run_button.on_click(on_run_button_click)

# Create and observe the breakout dropdown.
breakout_dropdown.observe(lambda change: update_breakout(dropdown.value), names='value')

# Create an observer on the ticker dropdown so that when its value changes,
# the breakout_dropdown resets to 'Channel', hiding the brand selectors.


# Create the ticker dropdown.
dropdown = widgets.Dropdown(
    options=possible_tickers,
    description='Ticker:',
    value='AAPL US Equity'
)

def on_ticker_change(change):
    if change['name'] == 'value':
        breakout_dropdown.value = 'Channel'
dropdown.observe(on_ticker_change, names='value')

# Action function to clear the brand checkboxes
def clear_selections(action):
    for box in brand_checkboxes:
        box.value=False

unselect_button = widgets.Button(description='Unselect All')
unselect_button.on_click(clear_selections)

# Wrap left and right outputs in VBoxes with a border.
left_box = widgets.VBox([left_out], layout=widgets.Layout(border='1px solid #CC5500', padding='5px', margin='5px'))
right_box = widgets.VBox([right_out], layout=widgets.Layout(border='1px solid #CC5500', padding='5px', margin='5px'))
columns = widgets.HBox([left_box, right_box])

# Display the widgets: ticker dropdown, run button, breakout dropdown, status, top output, breakout output, and the columns.
display(widgets.HBox([dropdown, run_button]), breakout_dropdown, top_out, status, breakout_out, columns)

# Initialize with the default ticker when app starts.
update_plots(dropdown.value)

Dropdown(description='Breakout By:', options=('Channel', 'Brands'), value='Channel')

Output()

HTML(value='')

Output()